In [1]:
import os
import cv2
import numpy as np
from imutils import paths

In [2]:
from sklearn.preprocessing import LabelBinarizer

In [3]:
from sklearn.model_selection import train_test_split

In [4]:
from keras.preprocessing.image import ImageDataGenerator

In [5]:
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import seaborn as sns
import pickle

import itertools

from sklearn.metrics import multilabel_confusion_matrix

In [6]:
datapath = r'/Users/valeriyavishnevskaya/Downloads/data'
outputmodel = r'/Users/valeriyavishnevskaya/Desktop/test/video_classification'
oplabelbinarizer = r'/Users/valeriyavishnevskaya/Desktop/test/video_classification/videoclassificationbinarizer'
epoch =25

In [7]:
sports_labels = set(['boxing','swimming',"table_tennis","badminton","football"])
print("Images loaded")
pathToImages = list(paths.list_images(datapath))
data = []
labels = []


for images in pathToImages:
    label = images.split(os.path.sep)[-2]
    if label not in sports_labels:
        continue
    image = cv2.imread(images)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, (224,224))
    data.append(image)
    labels.append(label)

Images loaded


In [8]:
data = np.array(data)
labels = np.array(labels)
#hot encoded values as 0,1,2,3,4

lb = LabelBinarizer()
labels = lb.fit_transform(labels)

In [9]:
(X_train, X_test, Y_train, Y_test) = train_test_split(data,labels,test_size=0.20, stratify=labels, random_state=42)

In [10]:
trainingAugmentation = ImageDataGenerator(
rotation_range=30,
zoom_range=0.15,
width_shift_range=0.2,
height_shift_range=0.2,
shear_range=0.15,
horizontal_flip=True,
fill_mode="nearest"
)
validationAugmentation = ImageDataGenerator()
mean = np.array([123.68, 116.779, 103.939], dtype="float32")
trainingAugmentation.mean = mean
validationAugmentation.mean = mean  

In [11]:
from keras.layers import Input
from keras.layers.pooling import AveragePooling2D
from keras.applications import ResNet50
from keras.layers.core import Dense
from keras.layers.core import Flatten

from keras.layers.core import Dropout
from keras.models import Model

In [12]:
baseModel = ResNet50(weights="imagenet", include_top= False, input_tensor=Input(shape=(224,224,3)))

headModel = baseModel.output
headModel = AveragePooling2D(pool_size=(7,7))(headModel)
headModel = Flatten(name="flatten")(headModel)
headModel = Dense(512, activation="relu")(headModel)
headModel = Dropout(0.5)(headModel)
headModel = Dense(len(lb.classes_), activation="softmax")(headModel)
model = Model(inputs=baseModel.input, outputs=headModel)

for basemodelLayers in baseModel.layers:
    basemodelLayers.trainable = False
  

In [13]:
from keras.optimizers import SGD
epoch=30

In [14]:
opt = SGD(learning_rate=0.0001, momentum=0.9, decay=1e-4//epoch)

In [15]:
model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=['accuracy','Precision','Recall'])

In [16]:
history = model.fit(
trainingAugmentation.flow(X_train, Y_train, batch_size=32),
steps_per_epoch=len(X_train) // 32,
validation_data=validationAugmentation.flow(X_test, Y_test),
validation_steps = len(X_test)// 32,
epochs=epoch)

<ipython-input-16-e450c13e2be9>:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(


Epoch 1/30
95/95 [==============================] - 492s 5s/step - loss: 1.6606 - accuracy: 0.3833 - precision: 0.4573 - recall: 0.2561 - val_loss: 0.8826 - val_accuracy: 0.6848 - val_precision: 0.8458 - val_recall: 0.5068
Epoch 2/30
95/95 [==============================] - 431s 5s/step - loss: 1.0354 - accuracy: 0.5952 - precision: 0.7023 - recall: 0.4763 - val_loss: 0.6262 - val_accuracy: 0.8139 - val_precision: 0.9126 - val_recall: 0.6807
Epoch 3/30
95/95 [==============================] - 460s 5s/step - loss: 0.7994 - accuracy: 0.7001 - precision: 0.7898 - recall: 0.6001 - val_loss: 0.5169 - val_accuracy: 0.8560 - val_precision: 0.9249 - val_recall: 0.7527
Epoch 4/30
95/95 [==============================] - 446s 5s/step - loss: 0.6900 - accuracy: 0.7499 - precision: 0.8266 - recall: 0.6636 - val_loss: 0.4561 - val_accuracy: 0.8804 - val_precision: 0.9358 - val_recall: 0.7921
Epoch 5/30
95/95 [==============================] - 461s 5s/step - loss: 0.6428 - accuracy: 0.7682 - precisi

In [ ]:
model.save(outputmodel)
lbinarizer.close()lbinarizer = open(r"Users/valeriyavishnevskaya/Desktop/test/video_classification/videoclassificationbinarizer.pickle","wb")
lbinarizer.write(pickle.dumps(lb))
lbinarizer.close()